# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from cachetools import LRUCache
from numpy.polynomial import Polynomial as polyeval

%matplotlib inline

ImportError: No module named 'cachetools'

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidYellowLeft.jpg')

# setup cachetools
cache = LRUCache(maxsize=4)
slope_cache = LRUCache(maxsize=2)

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def bgr_to_hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

def yellow_white_filter(img):
    # define range of white color in HSV
    lower_white = np.array([0,0,0], dtype=np.uint8)
    upper_white = np.array([200,200,200], dtype=np.uint8)

    # Threshold the HSV image to get only white colors
    mask = cv2.inRange(img, lower_white, upper_white)

    return mask

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def original_draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).

    Think about things like separating line segments by their
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)


def draw_lines(img, lines, color=[30, 208, 188], thickness=10):

    right_lane_x = []
    left_lane_x = []
    right_lane_y = []
    left_lane_y = []

    right_lengths = []
    left_lengths = []
    right_slopes = []
    left_slopes = []
    # set y
    y1_pt = 540
    y2_pt = 335

    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:

                # 1. get the slope and length
                slope = ((y2-y1)/(x2-x1))
                length = math.sqrt((x1-x2)**2 + (y1-y2)**2)

                # 2. run checks
                if 0.4 > slope > -0.4:
                    continue

                if slope > 0.9:
                    continue

                if slope < -0.9:
                    continue

                # 3. separate them between positive and negative slope
                if slope > 0:
                    # collect right lane x and y
                    right_lane_x.extend([x1, x2])
                    right_lane_y.extend([y1, y2])
                    right_slopes.append(slope)
                    right_lengths.append(length)

                else:
                    # collect left lane x and y
                    left_lane_x.extend([x1, x2])
                    left_lane_y.extend([y1, y2])
                    left_slopes.append(slope)
                    left_lengths.append(length)


        if right_lane_x and left_lane_x:

            # 13. get weighted slope
            right_slope = get_weighted_slope(right_slopes, right_lengths)
            left_slope = get_weighted_slope(left_slopes, left_lengths)

            if slope_cache.currsize > 0:
                previous_left_slope = slope_cache["left_slope"]
                previous_right_slope = slope_cache["right_slope"]

                right_change_percent = ((right_slope - previous_right_slope)/previous_right_slope)*100.0
                left_change_percent = ((left_slope - previous_left_slope)/previous_left_slope)*100.0

                slope_cache["left_slope"] = left_slope
                slope_cache["right_slope"] = right_slope

                right_change_percent = abs(right_change_percent)
                left_change_percent = abs(left_change_percent)

                # check to see if change in percentage is greater than 1.5%
                if right_change_percent > 1.5 or left_change_percent > 1.5:
                    show_previous_line(img, y1_pt, y2_pt)

                else:
                    # get polynomial fit
                    right_poly = polyeval.fit(np.array(right_lane_x), np.array(right_lane_y), 1)
                    left_poly = polyeval.fit(np.array(left_lane_x), np.array(left_lane_y), 1)

                    left_x1 = round((left_poly - y1_pt).roots()[0])
                    right_x1 = round((right_poly - y1_pt).roots()[0])

                    left_x2 = round((left_poly - y2_pt).roots()[0])
                    right_x2 = round((right_poly - y2_pt).roots()[0])

                    if cache.currsize > 0:
                        previous_left_x1 = cache["left_x1"]
                        previous_right_x1 = cache["right_x1"]
                        previous_left_x2 = cache["left_x2"]
                        previous_right_x2 = cache["right_x2"]

                        left_x1 = np.average([left_x1, previous_left_x1])
                        left_x2 = np.average([left_x2, previous_left_x2])
                        right_x1 = np.average([right_x1, previous_right_x1])
                        right_x2 = np.average([right_x2, previous_right_x2])

                    cache["left_x1"] = left_x1
                    cache["right_x1"] = right_x1
                    cache["left_x2"] = left_x2
                    cache["right_x2"] = right_x2

                    cv2.line(img, (int(left_x1), int(y1_pt)), (int(left_x2), int(y2_pt)), color, thickness)
                    cv2.line(img, (int(right_x1), int(y1_pt)), (int(right_x2), int(y2_pt)), color, thickness)
            else:
                slope_cache["left_slope"] = left_slope
                slope_cache["right_slope"] = right_slope

        else:
            show_previous_line(img, y1_pt, y2_pt)
    else:
        show_previous_line(img, y1_pt, y2_pt)


def get_top_percentile(points):
    top_limit = np.percentile(points, 80)
    bottom_limit = np.percentile(points, 20)
    new_points = []
    mean_point = np.mean(points)
    for point in points:
        if point > top_limit and point > bottom_limit:
            new_points.append(point)
        else:
            new_points.append(mean_point)

    return new_points

def show_previous_line(img, y1, y2):
    if cache.currsize > 1:
        left_x1 = cache["left_x1"]
        right_x1 = cache["right_x1"]
        left_x2 = cache["left_x2"]
        right_x2 = cache["right_x2"]

        cv2.line(img, (int(left_x1), int(y1)), (int(left_x2), int(y2)), color=[30, 208, 188], thickness=10)
        cv2.line(img, (int(right_x1), int(y1)), (int(right_x2), int(y2)), color=[30, 208, 188], thickness=10)

def get_weighted_slope(slopes, lengths):
    weighted_slope = sum(slopes[g] * lengths[g] for g in range(len(slopes))) / sum(lengths)
    return weighted_slope

def points_from_line(slope, x, intercept):
    y = slope*x + intercept
    return (int(math.floor(x)), int(math.floor(y)))
    
def get_y_intercept(m, x1, y1):
    b=y1-m*x1
    return b

def average_slope_intercept(x, y):
    A = np.vstack([x, np.ones(len(x))]).T
    m, b = np.linalg.lstsq(A, y)[0]
    return m, b

def get_mid_point(points):
        
    # TO DO: add weight here in terms of length
    # length = math.sqrt((x1-x2)**2 + (y1-y2)**2)

    return np.mean(points)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
test_images = os.listdir("test_images/")

print(test_images)

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

image = mpimg.imread("test_images/solidWhiteRight.jpg")

# bgr to hsv
    hsv = bgr_to_hsv(image)

    # filitered
    filtered = yellow_white_filter(hsv)

    # gaussian blur
    kernel_size = 7
    blur_gray = gaussian_blur(filtered, kernel_size)

    # canny
    low_threshold = 1
    high_threshold = 5
    edges = canny(blur_gray, low_threshold, high_threshold)

    # region of interest
    vertices = np.array([[(50, 550), (910,550), (510,330),(450, 330)]], dtype=np.int32)
    edges = region_of_interest(edges, vertices)

    # hough line
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/90
    threshold = 15
    min_line_len = 50
    max_line_gap = 30

    # Run Hough on edge detected image
    line_img = hough_lines(edges, rho, theta, threshold, min_line_len, max_line_gap)

    # weighted image
    weighted = weighted_img(line_img, image)
    return weighted

def yellow_white_filter(img):
    # define range of white color in HSV
    lower_white = np.array([0,0,0], dtype=np.uint8)
    upper_white = np.array([200,200,200], dtype=np.uint8)

    # Threshold the HSV image to get only white colors
    mask = cv2.inRange(img, lower_white, upper_white)

    return mask


plt.figure()
plt.imshow(lane_finder(image))    
plt.show()



## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
import imageio
imageio.plugins.ffmpeg.download()
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    return image

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(lane_finder)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.

### procedure

1. turned image to hsv
2. filtered it with white only
3. applied gaussian blur
4. applied canny edge detection
5. defined region of interest
6. setup hough line
7. laid hough line over initial image
8. in draw_lines

    8. got the slope and length of each line
    9. separated lines into right and left based on slope being negative or positive
    10. if slope was between 0.4 and -0.4, meaning too shallow, the line was discarded
    11. if the slope's absolute value was greater than 0.9 it was also discarded
    12. added x's and y's and slopes and lengths to it's own list
    13. calculated a weighted slope based on the length of the line
    14. calculated the change in percentage of the previous slope with current slop
    15. if previous slope was great than 1.5 percent for either lane, it was discarded
    16. then took the remaining x's in the right and left lane and ran a polynomial fit
    17. then averaged out the current x's with the previous x's so the transition is smoother
    18. then showed the line 
    19. if any of the parameters failed, the previously cached line was shown

### what to do better

- use EKF or UKF instead of all these if statements to correct all the errors that we're currently encoutering. throwing away a lot of errors.
- multi-dimensional poly fit for curved lines, currently only linear approx.
- find better hough line parameters
- make it work for rain, and dust, animals showing up, people
- not filtering properly for yellow color, should do that
- make it work for curved lines much better, doesn't work on challenge



## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(lane_finder)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))